In [12]:
from typing import Any, Dict, List
from langchain.chat_models import ChatOpenAI
# from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate # few_shot 은 모델들에게 더 나은 대답을 하도록 예제들을 주는 개념 (프롬프트로 모든걸 제어하기보다 효율적)
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector # 제공할 예제를 형식화하기
from langchain.prompts import load_prompt # 외부에서 프롬프트 파일 가져오기
from langchain.prompts.pipeline import PipelinePromptTemplate # 많은 프롬프트들을 하나로 합칠수 있게 하는거

#prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")

chat = ChatOpenAI(
    temperature=0.1,    # 숫자가 높을수록 랜덤함.
    streaming=True,     # 처리하는 과정을 살펴볼때 쓰는 옵션
    callbacks=[StreamingStdOutCallbackHandler()] # 처리하는 과정을 살펴볼때 쓰는 옵션2
    )    


intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonationg a {character}
    """
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
    """
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
    """
)

final = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
    """
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start)
]

full_prompt = PipelinePromptTemplate(
    final_prompt=final, 
    pipeline_prompts=prompts
)


full_prompt.format(
    character="Pirate",
    example_question="What is your location?",
    example_answer = "Arrrg! That is a secret!! Arg arg!!",
    question = "What is your fav food?"
)

chain = full_prompt | chat 

chain.invoke({
    "character":"Pirate",
    "example_question":"What is your location?",
    "example_answer":"Arrrg! That is a secret!! Arg arg!!",
    "question":"What is your fav food?"
})


Arrr matey! Me favorite food be a good ol' plate of salted fish and hardtack! Aye, nothing beats the taste of the sea! Arrr!

AIMessageChunk(content="Arrr matey! Me favorite food be a good ol' plate of salted fish and hardtack! Aye, nothing beats the taste of the sea! Arrr!")